In [1]:
from torch import cuda
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

embed_model_id = 'sentence-transformers/all-MiniLM-L6-v2'#'BAAI/bge-large-en'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

embed_model = HuggingFaceEmbeddings(
    model_name=embed_model_id,
    model_kwargs={'device': device},
    encode_kwargs={'device': device, 'batch_size': 32}
)

In [2]:
import os
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import PyPDFLoader
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain import HuggingFacePipeline
from langchain.text_splitter import CharacterTextSplitter
import textwrap
import sys

os.environ['HuggingFaceHub_API_Token']= 'YOUR KEY'

In [3]:
BASE_FOLDER = "RAW_DATA"
loaders = {
    '.pdf': DirectoryLoader(
        path=BASE_FOLDER,
        glob="**/*.pdf",
        show_progress=True
    ),
    '.md': DirectoryLoader(
        path=BASE_FOLDER,
        glob="**/*.md",
        show_progress=True
    ),
    '.txt': DirectoryLoader(
        path=BASE_FOLDER,
        glob="**/*.txt",
        show_progress=True
    ),
    # Add other file types and their respective loaders here
}

docs = []
for loader in loaders.values():
    docs.extend(loader.load())

 99%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████ | 262/264 [00:15<00:00, 16.80it/s]
0it [00:00, ?it/s]


In [4]:
text_splitter=CharacterTextSplitter(separator='\n', chunk_size=3200, chunk_overlap=20)
text_chunks=text_splitter.split_documents(docs)
#document_splitter=CharacterTextSplitter(separator='\n', chunk_size=1000, chunk_overlap=100)
#document_chunks=document_splitter.split_documents(docs)
len(text_chunks)
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')#'BAAI/bge-large-en')
vectordb = FAISS.from_documents(text_chunks,embedding=embeddings)#, persist_directory='PROCESSED_DATA/db')
#vectordb.persist()
vectordb.save_local("PROCESSED_DATA")
#FAISS.from_documents()